In [1]:
import torch
import torch.nn as nn
import torch.utils as utils
import torch.nn.init as init
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.utils as v_utils
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

# Set Hyperparameters

epoch = 100
batch_size = 300
learning_rate = 0.0005
num_gpus = 1
discrete_latent_size = 10
contin_latent_size = 2 

# Download Data & Set Data Loader(input pipeline)

mnist_train = dset.MNIST("./", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
train_loader = torch.utils.data.DataLoader(dataset=mnist_train,batch_size=batch_size,shuffle=True,drop_last=True)


def int_to_onehot(z_label):
    one_hot_array = np.zeros(shape=[len(z_label), discrete_latent_size])
    one_hot_array[np.arange(len(z_label)), z_label] = 1
    return one_hot_array


class Generator(nn.Module):
    def __init__(self):
        super(Generator,self).__init__()
        self.layer1 = nn.Linear(112,7*7*256)                    # [batch,110] -> [batch,7*7*256]
        self.layer2 = nn.Sequential(OrderedDict([
                ('conv1', nn.ConvTranspose2d(256,128,3,2,1,1)), # [batch,256,7,7] -> [batch,128,14,14]
                ('relu1', nn.LeakyReLU()),
                ('bn1', nn.BatchNorm2d(128)),
                ('conv2', nn.ConvTranspose2d(128,64,3,1,1)),    # [batch,128,14,14] -> [batch,64,14,14]
                ('relu2', nn.LeakyReLU()),
                ('bn2', nn.BatchNorm2d(64))
            ]))
        self.layer3 = nn.Sequential(OrderedDict([
                ('conv3',nn.ConvTranspose2d(64,16,3,1,1)),      # [batch,64,14,14] -> [batch,16,14,14]
                ('relu3',nn.LeakyReLU()),
                ('bn3',nn.BatchNorm2d(16)),
                ('conv4',nn.ConvTranspose2d(16,1,3,2,1,1)),     # [batch,16,14,14] -> [batch,1,28,28]
                ('relu4',nn.LeakyReLU())
            ]))

    def forward(self,z):
        out = self.layer1(z)
        out = out.view(batch_size//num_gpus,256,7,7)
        out = self.layer2(out)
        out = self.layer3(out)
        return out


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator,self).__init__()
        self.layer1 = nn.Sequential(OrderedDict([
                ('conv1',nn.Conv2d(1,16,3,padding=1)),   # [batch,1,28,28] -> [batch,16,28,28]
                ('relu1',nn.LeakyReLU()),
                ('bn1',nn.BatchNorm2d(16)),
                ('conv2',nn.Conv2d(16,32,3,padding=1)),  # [batch,16,28,28] -> [batch,32,28,28]
                ('relu2',nn.LeakyReLU()),
                ('bn2',nn.BatchNorm2d(32)),
                ('max1',nn.MaxPool2d(2,2))               # [batch,32,28,28] -> [batch,32,14,14]
            ]))
        self.layer2 = nn.Sequential(OrderedDict([
                ('conv3',nn.Conv2d(32,64,3,padding=1)),  # [batch,32,14,14] -> [batch,64,14,14] 
                ('relu3',nn.LeakyReLU()),
                ('bn3',nn.BatchNorm2d(64)),	
                ('max2',nn.MaxPool2d(2,2)),	             # [batch,64,14,14] -> [batch,64,7,7]
                ('conv4',nn.Conv2d(64,128,3,padding=1)), # [batch,64,7,7] -> [batch,128,7,7]
                ('relu4',nn.LeakyReLU())
            ]))
        self.fc = nn.Sequential(
                nn.Linear(128*7*7,1),
                nn.Sigmoid()
            )
        self.fc2 = nn.Sequential(
                nn.Linear(128*7*7,10),
                nn.LeakyReLU(),
            )
        self.fc3 = nn.Sequential(
                nn.Linear(128*7*7,2),
                nn.LeakyReLU(),
            )

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(batch_size//num_gpus, -1)
        
        output = self.fc(out)
        onehot = self.fc2(out)
        contin = self.fc3(out)
        
        return output,onehot,contin
    
# put class instance on multi gpu

generator = nn.DataParallel(Generator(),device_ids=[0])
discriminator = nn.DataParallel(Discriminator(),device_ids=[0])

# put labels on multi gpu

ones_label = Variable(torch.ones(batch_size,1)).cuda()
zeros_label = Variable(torch.zeros(batch_size,1)).cuda()

# loss function and optimizer 
# this time, use LSGAN loss(https://arxiv.org/abs/1611.04076v2)

loss_func = nn.MSELoss()
gen_optim = torch.optim.Adam(generator.parameters(), lr=learning_rate)
dis_optim = torch.optim.Adam(discriminator.parameters(), lr=learning_rate)

# model restore

try:
    generator, discriminator = torch.load('./model/infogan.pkl')
    print("\n--------model restored--------\n")
except:
    print("\n--------model not restored--------\n")
    pass

# train 

for i in range(epoch):
    for j,(image,label) in enumerate(train_loader):
        
        # put image & label on gpu
        image = Variable(image).cuda()
        label = torch.from_numpy(int_to_onehot(label.numpy()))
        label = Variable(label.type_as(torch.FloatTensor())).cuda()
    
        # generator 
        for k in range(2):
            z_random = np.random.rand(batch_size,100)
            z_onehot = np.random.randint(0, 10, size=batch_size)
            z_contin = np.random.uniform(-2,2,size=[batch_size,2])
            
            # change first 10 labels from random to 0~9          
            for l in range(10):
                z_onehot[l]=l
            
            # preprocess z
            z_label_onehot = int_to_onehot(z_onehot)
            z_concat = np.concatenate([z_random, z_label_onehot,z_contin], axis=1)
            z = Variable(torch.from_numpy(z_concat).type_as(torch.FloatTensor())).cuda()
            z_label_onehot = Variable(torch.from_numpy(z_label_onehot).type_as(torch.FloatTensor())).cuda()
            z_label_contin = Variable(torch.from_numpy(z_contin).type_as(torch.FloatTensor())).cuda()

            # calculate loss and apply gradients
            # gen_loss = gan loss(fake) + categorical loss
            gen_optim.zero_grad()
            gen_fake = generator.forward(z)
            dis_fake,onehot_fake,contin_fake = discriminator.forward(gen_fake)
            gen_loss = torch.sum(loss_func(dis_fake,ones_label)) \
                     + discrete_latent_size * torch.sum(loss_func(onehot_fake,z_label_onehot))\
                     + 1 * torch.sum(loss_func(contin_fake,z_label_contin))
                
            gen_loss.backward(retain_variables=True)
            gen_optim.step()

        # discriminator
        # dis_loss = gan_loss(fake & real) + categorical loss
        dis_optim.zero_grad()
        dis_real, onehot_real,contin_real = discriminator.forward(image)
        dis_loss = torch.sum(loss_func(dis_fake,zeros_label))\
                 + torch.sum(loss_func(dis_real,ones_label))\
                 + discrete_latent_size * torch.sum(loss_func(onehot_real,label))\
                 + 1 * torch.sum(loss_func(contin_real,z_label_contin))
                
        dis_loss.backward()
        dis_optim.step()
    
        # model save
        if j % 100 == 0:
            torch.save([generator,discriminator],'./model/infogan.pkl')

            # print loss and image save
            print("{}th iteration gen_loss: {} dis_loss: {}".format(i,gen_loss.data,dis_loss.data))
            v_utils.save_image(gen_fake.data[0:20],"./result/gen_{}_{}.png".format(i,j), nrow=5)



--------model not restored--------



/usr/local/lib/python3.5/dist-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type Generator. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.5/dist-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type Discriminator. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


0th iteration gen_loss: 
 2.4904
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 2.9399
[torch.cuda.FloatTensor of size 1 (GPU 0)]

0th iteration gen_loss: 
 3.1848
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.4924
[torch.cuda.FloatTensor of size 1 (GPU 0)]

1th iteration gen_loss: 
 3.1848
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.4114
[torch.cuda.FloatTensor of size 1 (GPU 0)]

1th iteration gen_loss: 
 2.9469
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.3457
[torch.cuda.FloatTensor of size 1 (GPU 0)]

2th iteration gen_loss: 
 3.3219
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.5941
[torch.cuda.FloatTensor of size 1 (GPU 0)]



KeyboardInterrupt: 

In [2]:
for i in range(10):
    z_random = np.random.rand(batch_size,100)
    z_onehot = np.random.randint(0, 10, size=batch_size)
    z_contin = np.random.uniform(-2,2,size=[batch_size,2])

    # change first 10 labels from random to 0~9          
    for l in range(40):
        z_onehot[l]=i
        
        if l <= 20: 
            z_contin[l,0]= (l-10)/5
            z_contin[l,1]= 0 
        else:      
            z_contin[l,0]= 0
            z_contin[l,1]= (l-30)/5
            
    print(z_contin)

    # preprocess z
    z_label_onehot = int_to_onehot(z_onehot)
    z_concat = np.concatenate([z_random, z_label_onehot,z_contin], axis=1)
    z = Variable(torch.from_numpy(z_concat).type_as(torch.FloatTensor())).cuda()
    
    gen_fake = generator.forward(z)
    
    v_utils.save_image(gen_fake.data[0:40],"./result/gen_{}_{}.png".format(i,j), nrow=20)

[[-2.          0.        ]
 [-1.8         0.        ]
 [-1.6         0.        ]
 [-1.4         0.        ]
 [-1.2         0.        ]
 [-1.          0.        ]
 [-0.8         0.        ]
 [-0.6         0.        ]
 [-0.4         0.        ]
 [-0.2         0.        ]
 [ 0.          0.        ]
 [ 0.2         0.        ]
 [ 0.4         0.        ]
 [ 0.6         0.        ]
 [ 0.8         0.        ]
 [ 1.          0.        ]
 [ 1.2         0.        ]
 [ 1.4         0.        ]
 [ 1.6         0.        ]
 [ 1.8         0.        ]
 [ 2.          0.        ]
 [ 0.         -1.8       ]
 [ 0.         -1.6       ]
 [ 0.         -1.4       ]
 [ 0.         -1.2       ]
 [ 0.         -1.        ]
 [ 0.         -0.8       ]
 [ 0.         -0.6       ]
 [ 0.         -0.4       ]
 [ 0.         -0.2       ]
 [ 0.          0.        ]
 [ 0.          0.2       ]
 [ 0.          0.4       ]
 [ 0.          0.6       ]
 [ 0.          0.8       ]
 [ 0.          1.        ]
 [ 0.          1.2       ]
 